In [ ]:
import pandas as pd

# Load the sample data
df = pd.read_csv("1sample_providers.csv", dtype=str)

# Keep only individuals (Entity Type Code = "1")
df_people = df[df["Entity Type Code"] == "1"].copy()

# Select relevant columns for the providers table
providers = df_people[[
    "NPI",
    "Provider First Name",
    "Provider Last Name (Legal Name)",
    "Provider Middle Name",
    "Provider Credential Text",
    "Provider First Line Business Mailing Address",
    "Provider Second Line Business Mailing Address",
    "Provider Business Mailing Address City Name",
    "Provider Business Mailing Address State Name",
    "Provider Business Mailing Address Postal Code"
]].copy()

# Rename to match the database schema
providers.columns = [
    "npi",
    "first_name",
    "last_name",
    "middle_name",
    "credential",
    "address_1",
    "address_2",
    "city",
    "state",
    "zip"
]

# Save as CSV
providers.to_csv("providers.csv", index=False) # NPI as PK


In [2]:
import pandas as pd

# Load taxonomy data
df_tax = pd.read_csv("1nucc_taxonomy_251.csv", dtype=str)

# Keep relevant columns only
taxonomy = df_tax[[
    "Code",
    "Grouping",
    "Classification",
    "Specialization",
    "Definition",
    "Display Name"
]].copy()

# Rename for database-style names
taxonomy.columns = [
    "code",
    "grouping",
    "classification",
    "specialization",
    "definition",
    "display_name"
]

# Save to clean file
taxonomy.to_csv("taxonomy.csv", index=False)


In [3]:
import pandas as pd

# Load providers sample
df = pd.read_csv("1sample_providers.csv", dtype=str)
df = df[df["Entity Type Code"] == "1"].copy()

# Build list of records (id will be auto-added later)
records = []

for i in range(1, 16):
    tax_col = f"Healthcare Provider Taxonomy Code_{i}"
    prim_col = f"Healthcare Provider Primary Taxonomy Switch_{i}"
    
    if tax_col in df.columns and prim_col in df.columns:
        chunk = df[["NPI", tax_col, prim_col]].copy()
        chunk.columns = ["provider_npi", "taxonomy_code", "primary_raw"]
        chunk = chunk[chunk["taxonomy_code"].notna()]
        chunk["primary"] = chunk["primary_raw"] == "Y"
        chunk = chunk[["provider_npi", "taxonomy_code", "primary"]]
        records.append(chunk)

# Combine all taxonomy connections into one DataFrame
provider_taxonomies = pd.concat(records, ignore_index=True)

# Add ID column
provider_taxonomies.insert(0, "id", range(1, len(provider_taxonomies) + 1))

# Save to CSV
provider_taxonomies.to_csv("provider_taxonomies.csv", index=False)
